<a href="https://colab.research.google.com/github/PilaData/Aircraft-dents-detection-/blob/master/Mask_scoring_Rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.dropbox.com/s/8p0v7n4qf86y0c0/Data.zip?dl=0 -O Data.zip
!unzip Data.zip

--2020-08-18 15:39:22--  https://www.dropbox.com/s/8p0v7n4qf86y0c0/Data.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8p0v7n4qf86y0c0/Data.zip [following]
--2020-08-18 15:39:22--  https://www.dropbox.com/s/raw/8p0v7n4qf86y0c0/Data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4723d5dcedf87705086f3aec07.dl.dropboxusercontent.com/cd/0/inline/A9sHRgw3QC0nXkTtKu36SByQJhID7KwlpFfFER0tXzUw61NScHT-Nv_plqJmDmixsFnFdFNg13Xa8dju8K7ee1sg-h8E_kK8EeaCyuy7fSYyLlXy7Uie-51S6s8v8pwH3d4/file# [following]
--2020-08-18 15:39:22--  https://uc4723d5dcedf87705086f3aec07.dl.dropboxusercontent.com/cd/0/inline/A9sHRgw3QC0nXkTtKu36SByQJhID7KwlpFfFER0tXzUw61NScHT-Nv_plqJmDmixsFnFdFNg13Xa8dju8K7ee1sg-h8E_kK8EeaCyuy7fSYyLlXy7Uie-51S6

In [ ]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.6.0+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.7.0+cu101)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-lim8amkh
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-lim8amkh
     |████████████████████████████████| 276kB 5.9MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1-cp36-none-any.whl size=47499 sha256=8cf9b0441148914d29d04ecccc3949ddf8f196310efde1fd7b6047f868c39e43
  Stored in directory: /tmp/pip-ephem-wheel-cache-fh82v2px/wheels/48/53/79/3c6485543a4455a0006f5db590ab9957622b6227011941de06
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=0f031941b9890a30ef749d91439f6920a232fee43be973f882a48c16c8d034d8
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully

In [ ]:

import torch, torchvision
import numpy as np
import pandas as pd
import random
import json
import cv2
import os
os.chdir('/content/detectron2_repo')
import detectron2
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.evaluation.coco_evaluation import COCOEvaluator

import itertools
from itertools import groupby
from google.colab.patches import cv2_imshow

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskutil
from pycocotools import mask as maskUtils

In [ ]:
# register training and test datasets to Detectron2
register_coco_instances("T", {}, "/content/Data/train/annotation.json", "/content/Data/train")
register_coco_instances("V", {}, "/content/Data/val/annotation.json", "/content/Data/val")

# metadata
train_metadata = MetadataCatalog.get("T")
test_metadata = MetadataCatalog.get("V")

# dataset dictionary
train_dataset_dicts = DatasetCatalog.get("T")
test_dataset_dicts = DatasetCatalog.get("V")




WARNING [08/18 15:42:13 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/18 15:42:13 d2.data.datasets.coco]: Loaded 96 images in COCO format from /content/Data/train/annotation.json
WARNING [08/18 15:42:13 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/18 15:42:13 d2.data.datasets.coco]: Loaded 16 images in COCO format from /content/Data/val/annotation.json


In [ ]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import glob

import os
import ntpath
import numpy as np
import cv2
import random
import itertools
import pandas as pd
from tqdm import tqdm
import urllib
import json
import PIL.Image as Image

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
cfg = get_cfg()

# load ImageNet pretrained weights
cfg.merge_from_file("/content/detectron2_repo/configs/quick_schedules/cascade_mask_rcnn_R_50_FPN_inference_acc_test.yaml")




In [ ]:
cfg.DATASETS.TRAIN = ("T",)
cfg.DATASETS.TEST = ("V",)
cfg.DATALOADER.NUM_WORKERS = 1

In [ ]:
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1000, 2000)
cfg.SOLVER.GAMMA = 0.05

In [ ]:

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 8
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.EVAL_PERIOD = 500

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[08/18 15:42:24 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_480dd8.pkl: 288MB [00:04, 63.8MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.2.cls_score.weight' to the mo

[08/18 15:42:34 d2.engine.train_loop]: Starting training from iteration 0


/content/detectron2_repo/detectron2/structures/masks.py:345: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()
/content/detectron2_repo/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


[08/18 15:42:44 d2.utils.events]:  eta: 0:16:36  iter: 19  total_loss: 4.774  loss_cls_stage0: 0.654  loss_box_reg_stage0: 0.487  loss_cls_stage1: 0.694  loss_box_reg_stage1: 0.696  loss_cls_stage2: 0.718  loss_box_reg_stage2: 0.612  loss_mask: 0.694  loss_rpn_cls: 0.084  loss_rpn_loc: 0.011  time: 0.5002  data_time: 0.0946  lr: 0.000020  max_mem: 4468M
[08/18 15:42:54 d2.utils.events]:  eta: 0:16:14  iter: 39  total_loss: 4.336  loss_cls_stage0: 0.594  loss_box_reg_stage0: 0.392  loss_cls_stage1: 0.609  loss_box_reg_stage1: 0.539  loss_cls_stage2: 0.604  loss_box_reg_stage2: 0.593  loss_mask: 0.670  loss_rpn_cls: 0.089  loss_rpn_loc: 0.010  time: 0.4951  data_time: 0.0790  lr: 0.000040  max_mem: 4468M
[08/18 15:43:04 d2.utils.events]:  eta: 0:16:06  iter: 59  total_loss: 4.137  loss_cls_stage0: 0.516  loss_box_reg_stage0: 0.488  loss_cls_stage1: 0.531  loss_box_reg_stage1: 0.636  loss_cls_stage2: 0.501  loss_box_reg_stage2: 0.675  loss_mask: 0.629  loss_rpn_cls: 0.058  loss_rpn_loc: 0

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%load_ext tensorboard



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir output

In [ ]:
# register training and test datasets to Detectron2
register_coco_instances("D", {}, "/content/Data/Test/annotation.json", "/content/Data/Test/")


# metadata
v_metadata = MetadataCatalog.get("D")


# dataset dictionary
v_dataset_dicts = DatasetCatalog.get("D")



In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("D", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
import random
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow

for d in random.sample(v_dataset_dicts, 20):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=v_metadata, 
                   scale=0.8, 
                   
      )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [31]:
evaluator = COCOEvaluator("D", cfg, False, output_dir="/content/detectron2_repo/output")
val_loader = build_detection_test_loader(cfg, "D")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [08/18 17:21:50 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/18 17:21:50 d2.data.datasets.coco]: Loaded 22 images in COCO format from /content/Data/Test/annotation.json
[08/18 17:21:50 d2.data.common]: Serializing 22 elements to byte tensors and concatenating them all ...
[08/18 17:21:50 d2.data.common]: Serialized dataset takes 0.03 MiB
[08/18 17:21:50 d2.data.dataset_mapper]: Augmentations used in training: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/18 17:21:50 d2.evaluation.evaluator]: Start inference on 22 images
[08/18 17:21:54 d2.evaluation.evaluator]: Inference done 11/22. 0.0902 s / img. ETA=0:00:05
[08/18 17:21:55 d2.evaluation.evaluator]: Total inference time: 0:00:04.291723 (0.252454 s / img per device, on 1 devices)
[08/18 17:21:55 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:01 (0.087248 s / img per device, on 1 devices)
[0

OrderedDict([('bbox',
              {'AP': 61.16784317758873,
               'AP50': 81.00243676468726,
               'AP75': 77.53446397271307,
               'APl': 75.56848184818482,
               'APm': 39.459276284771335,
               'APs': 65.04950495049505}),
             ('segm',
              {'AP': 57.4341158054299,
               'AP50': 81.00243676468726,
               'AP75': 77.53446397271307,
               'APl': 78.43684368436844,
               'APm': 26.980198019801982,
               'APs': 60.0})])